# Scripts to auto-generate nlu.yml and stories.yml

In [48]:
root = './RASA_ConceptNet5/data/'
processed = './Data/processing/Processed_Airbnb/'

### Import

In [49]:
import pandas as pd 

## Generate nlu.yml

### Gather FAQs

In [50]:
FAQ1 = pd.read_csv(processed+'FAQ_1.json')
FAQ2 = pd.read_csv(processed+'FAQ_2.json')
FAQ3 = pd.read_csv(processed+'FAQ_3.json')


FAQ =  pd.concat([FAQ1, FAQ2, FAQ3], axis=0)

FAQ.head(1)

,question,answer,listing_id,context
0,Where did I find the place from?,central station,2818.0,I didnt have any trouble finding the place fro...


### Add intent type

In [51]:
FAQ['intent'] = 'find_listing_by_review_context'
FAQ['type'] = 'intent'

In [52]:
FAQ.head(5)

,question,answer,listing_id,context,intent,type
0,Where did I find the place from?,central station,2818.0,I didnt have any trouble finding the place fro...,find_listing_by_review_context,intent
1,Who is really cool?,daniel,2818.0,Daniel is really cool.,find_listing_by_review_context,intent
2,What was in the room for you to use?,maps,2818.0,He had maps and a lonely planet guide book in ...,find_listing_by_review_context,intent
3,What is the best book you could ever need?,guidebooks,2818.0,"His place is extremely clean, and he provides ...",find_listing_by_review_context,intent
4,What is the best thing about the place?,comfy bed,2818.0,"His place is extremely clean, and he provides ...",find_listing_by_review_context,intent


### Append entity and entity value to the intent.

In [53]:
#Remove ? from the question 
#Append entity value and entity at the end and append '?' => [answer](wordid)?

FAQ['question'] = FAQ['question'].str.replace(r'?', ' ',regex=True)
FAQ.head(5)

,question,answer,listing_id,context,intent,type
0,Where did I find the place from,central station,2818.0,I didnt have any trouble finding the place fro...,find_listing_by_review_context,intent
1,Who is really cool,daniel,2818.0,Daniel is really cool.,find_listing_by_review_context,intent
2,What was in the room for you to use,maps,2818.0,He had maps and a lonely planet guide book in ...,find_listing_by_review_context,intent
3,What is the best book you could ever need,guidebooks,2818.0,"His place is extremely clean, and he provides ...",find_listing_by_review_context,intent
4,What is the best thing about the place,comfy bed,2818.0,"His place is extremely clean, and he provides ...",find_listing_by_review_context,intent


In [54]:
FAQ['question'] = FAQ['question'].astype(str) +'['+FAQ['answer'].astype(str)+'](wordid)?'

In [55]:
FAQ.head(5)

,question,answer,listing_id,context,intent,type
0,Where did I find the place from [central stati...,central station,2818.0,I didnt have any trouble finding the place fro...,find_listing_by_review_context,intent
1,Who is really cool [daniel](wordid)?,daniel,2818.0,Daniel is really cool.,find_listing_by_review_context,intent
2,What was in the room for you to use [maps](wor...,maps,2818.0,He had maps and a lonely planet guide book in ...,find_listing_by_review_context,intent
3,What is the best book you could ever need [gui...,guidebooks,2818.0,"His place is extremely clean, and he provides ...",find_listing_by_review_context,intent
4,What is the best thing about the place [comfy ...,comfy bed,2818.0,"His place is extremely clean, and he provides ...",find_listing_by_review_context,intent


### Auto-generate NLU intents

In [56]:
nlus = FAQ.groupby('intent')['question'].apply(list).to_dict()

In [57]:
#;TYPE;INTENT;VALUE
f = open(root+"nlu.yml","a")
# f.write('version: "2.0"\n')
# f.write("nlu:\n")
for intent in nlus:
    f.write("- intent: {}\n".format(intent))
    f.write("  examples: |\n")
    for value in nlus[intent]:
        f.write(" - {}\n".format(value))
f.close()